In [1]:
from tensorflow import keras 
from tensorflow.keras import layers
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
import pickle
import pandas as pd


epochs = 10
dotrain = True
dotest = True

In [2]:
Y = pd.DataFrame()


prefix = 'C:\\Projects\\Professional_Certificate_in_ML_and_AI\\Module23\\Notebooks_starter\\traffic\\2018\\flow_station_1209204_2018_'
for i in range(1,12):
    filename = '{}{:02d}'.format(prefix,i)
    with open(filename,'rb') as f:
        X = pickle.load(f)
        X.set_index(pd.to_datetime(X['Timestamp']),inplace=True)
        X.drop(columns=['Timestamp'],inplace=True)
        Y = Y.append(X)
        
prefix = 'C:\\Projects\\Professional_Certificate_in_ML_and_AI\\Module23\\Notebooks_starter\\traffic\\2019\\flow_station_1209204_2019_'
for i in range(1,12):
    filename = '{}{:02d}'.format(prefix,i)
    with open(filename,'rb') as f:
        X = pickle.load(f)
        X.set_index(pd.to_datetime(X['Timestamp']),inplace=True)
        X.drop(columns=['Timestamp'],inplace=True)
        Y = Y.append(X)
        
        


C:\Users\Plamen\AppData\Local\Temp\ipykernel_24452\235935179.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Y = Y.append(X)
C:\Users\Plamen\AppData\Local\Temp\ipykernel_24452\235935179.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Y = Y.append(X)
C:\Users\Plamen\AppData\Local\Temp\ipykernel_24452\235935179.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Y = Y.append(X)
C:\Users\Plamen\AppData\Local\Temp\ipykernel_24452\235935179.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Y = Y.append(X)
C:\Users\Plamen\AppData\Local\Temp\ipykernel_24452\235935179.py:11: FutureWarning: The frame.append method is deprecated and

In [3]:
raw_data = Y['Total Flow'].values
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 6990
num_val_samples: 3495
num_test_samples: 3495


In [4]:
raw_data.shape

(13980,)

In [5]:
sampling_rate = 1
sequence_length = 24*7 
delay = sequence_length
batch_size = 256
  
train_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)
  
val_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)
  
test_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

In [6]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    print(samples, targets)
    break

samples shape: (256, 168)
targets shape: (256,)
tf.Tensor(
[[ 3637.  3955.  3464. ...  5876.  4618.  3402.]
 [ 3955.  3464.  2810. ...  4618.  3402.  2821.]
 [ 3464.  2810.  2659. ...  3402.  2821.  2216.]
 ...
 [10108. 10658. 10813. ...  9214.  9591. 10094.]
 [10658. 10813. 10804. ...  9591. 10094. 10079.]
 [10813. 10804.  9810. ... 10094. 10079. 10585.]], shape=(256, 168), dtype=float64) tf.Tensor(
[ 2821.  2216.  1954.  2158.  2967.  4828.  7552.  9820.  9424.  9595.
  8641.  8940.  9086.  9173.  9855. 10251.  9974.  9015.  9045.  8030.
  6390.  5958.  5375.  4298.  3510.  2705.  2463.  2065.  2643.  4362.
  7268.  9608.  9833.  8945.  8597.  9178.  9145.  9471.  9411.  9405.
 10018.  9821.  8785.  7544.  5592.  5541.  4314.  3072.  2090.  1466.
  1510.  1313.  2429.  4509.  7682. 10091. 10267.  9668.  9258.  9780.
  9817. 10259. 10937. 11364. 11186. 10151.  8779.  8630.  6769.  5734.
  4885.  3446.  2288.  1643.  1881.  1335.  2306.  4569.  7473. 10275.
 10176. 10154.  9802.  9813.

In [7]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

In [8]:
def evaluate_naive_method(dataset):
    total_abs_err = 0. 
    samples_seen = 0 
    for samples, targets in dataset:
        preds = samples[:, -1] * std + mean
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err / samples_seen
  
print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}") 
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}")

Validation MAE: 20350927.57
Test MAE: 19979499.63


# DENSE

In [9]:
if dotrain:

  model_dense = keras.models.Sequential([ keras.Input(shape=sequence_length), keras.layers.Flatten(), keras.layers.Dense(16, activation="relu"), keras.layers.Dense(1) ])

#   callbacks = [ keras.callbacks.ModelCheckpoint("drive/MyDrive/Colab Notebooks/jena_dense.keras",
#                                       save_best_only=True) ] 

  model_dense.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_dense = model_dense.fit(train_dataset, epochs=1, validation_data=val_dataset)

27/27 [==============================] - 1s 13ms/step - loss: 9173079.0000 - mae: 2281.0916 - val_loss: 4047348.0000 - val_mae: 1606.5944


In [10]:
if dotrain:

  model_1Dconv = keras.models.Sequential([ keras.Input(shape=(sequence_length,1)), layers.Conv1D(8, 16, activation="relu"), layers.GlobalAveragePooling1D(), layers.Dense(1) ])

  model_1Dconv.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
  history_1Dconv = model_1Dconv.fit(train_dataset, epochs=1, validation_data=val_dataset)

27/27 [==============================] - 1s 15ms/step - loss: 8701356.0000 - mae: 2637.0139 - val_loss: 9324057.0000 - val_mae: 2771.8225


In [11]:
if dotrain:

  model_simprnn = keras.models.Sequential([ keras.Input(shape=(sequence_length,1)), layers.SimpleRNN(16, return_sequences=False)  ])                 

  model_simprnn.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_simprnn = model_simprnn.fit(train_dataset, epochs=1, validation_data=val_dataset)

27/27 [==============================] - 1s 24ms/step - loss: 63237812.0000 - mae: 7399.2139 - val_loss: 57530964.0000 - val_mae: 6959.6909


In [12]:
sequence_length

168

In [13]:
if dotrain:

  model_lstm = keras.models.Sequential([ keras.Input(shape=(sequence_length, 1)), layers.LSTM(16), layers.Dense(1) ])                 
  
  model_lstm.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_lstm = model_lstm.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

Epoch 1/10
27/27 [==============================] - 3s 65ms/step - loss: 63213056.0000 - mae: 7397.5405 - val_loss: 57506168.0000 - val_mae: 6957.9097
Epoch 2/10
27/27 [==============================] - 1s 52ms/step - loss: 63210128.0000 - mae: 7397.3423 - val_loss: 57503596.0000 - val_mae: 6957.7246
Epoch 3/10
27/27 [==============================] - 1s 50ms/step - loss: 63207404.0000 - mae: 7397.1582 - val_loss: 57501040.0000 - val_mae: 6957.5415
Epoch 4/10
27/27 [==============================] - 1s 50ms/step - loss: 63204688.0000 - mae: 7396.9756 - val_loss: 57498480.0000 - val_mae: 6957.3574
Epoch 5/10
27/27 [==============================] - 1s 51ms/step - loss: 63201964.0000 - mae: 7396.7925 - val_loss: 57495928.0000 - val_mae: 6957.1743
Epoch 6/10
27/27 [==============================] - 2s 56ms/step - loss: 63199256.0000 - mae: 7396.6079 - val_loss: 57493372.0000 - val_mae: 6956.9902
Epoch 7/10
27/27 [==============================] - 1s 54ms/step - loss: 63196532.0000 - mae: 